In [1]:
#my approch
import os
import cv2
import numpy as np
import pandas as pd
from skimage.metrics import structural_similarity as ssim
from skimage.measure import shannon_entropy

import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.measure import shannon_entropy

def compute_metrics(original, enhanced):
    """
    Compute PSNR, SSIM, and Shannon Entropy between the original and enhanced images.
    
    Parameters:
        original (np.array): Original image (BGR, uint8).
        enhanced (np.array): Enhanced image (BGR, uint8).
    
    Returns:
        tuple: (PSNR, SSIM, Entropy)
    """
    # Ensure both images have the same size (resize enhanced if necessary)
    if original.shape != enhanced.shape:
        enhanced = cv2.resize(enhanced, (original.shape[1], original.shape[0]))
    
    # PSNR computation
    psnr_value = cv2.PSNR(original, enhanced)
    
    # Determine an appropriate win_size for SSIM based on the image dimensions.
    h, w = original.shape[:2]
    min_dim = min(h, w)
    
    # If the smallest dimension is less than 3 pixels, SSIM is unreliable.
    if min_dim < 3:
        ssim_value = np.nan
    else:
        # Choose a win_size that is odd and does not exceed min_dim.
        if min_dim < 7:
            win_size = min_dim if (min_dim % 2 == 1) else (min_dim - 1)
        else:
            win_size = 7  # default win_size for larger images
        
        # Compute SSIM using the computed win_size and specifying channel_axis=-1 for color images.
        try:
            ssim_value, _ = ssim(original, enhanced, full=True, channel_axis=-1, win_size=win_size)
        except ValueError as e:
            print(f"Error computing SSIM for image of size ({h}, {w}) with win_size={win_size}: {e}")
            ssim_value = np.nan

    # Compute Shannon entropy on the enhanced image.
    # Convert the enhanced image to grayscale.
    enhanced_gray = cv2.cvtColor(enhanced, cv2.COLOR_BGR2GRAY)
    entropy_value = shannon_entropy(enhanced_gray)
    
    return psnr_value, ssim_value, entropy_value



def process_folders(original_folder, enhanced_folder, output_csv=r"D:\mit-5k-subset\my_enhancement_metrics_partd.csv"):
    valid_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')
    results = []

    for filename in os.listdir(original_folder):
        if filename.lower().endswith(valid_extensions):
            orig_path = os.path.join(original_folder, filename)

            # Remove 'normal' prefix and add 'low' prefix
            basename = filename.replace('normal', 'low', 1)  # replace only the first occurrence
            enh_path = os.path.join(enhanced_folder, basename)

            if not os.path.exists(enh_path):
                print(f"Enhanced image for {filename} not found (expected {basename}). Skipping.")
                continue

            original_img = cv2.imread(orig_path)
            enhanced_img = cv2.imread(enh_path)

            if original_img is None or enhanced_img is None:
                print(f"Error reading {filename}. Skipping.")
                continue

            psnr_val, ssim_val, entropy_val = compute_metrics(original_img, enhanced_img)
            results.append({
                'Filename': filename,  # you can also use a common name like basename if you prefer
                'My PSNR': psnr_val,
                'My SSIM': ssim_val,
                'My Entropy': entropy_val
            })
            print(f"Processed {filename}: PSNR={psnr_val:.2f}, SSIM={ssim_val:.3f}, Entropy={entropy_val:.2f}")

    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")
    return df




    # Replace these with the paths to your folders.
original_folder = r"D:\mit-5k-subset\raw"
enhanced_folder = r"D:\mit-5k-subset\d_enhanced"
    
    # Process the folders and save metrics to a CSV.
metrics_df1 = process_folders(original_folder, enhanced_folder)


Processed a0034-LSYD4O2202.jpg: PSNR=12.55, SSIM=0.271, Entropy=7.87
Processed a0035-dgw_048.jpg: PSNR=12.28, SSIM=0.467, Entropy=7.56
Processed a0291-IMG_0115.jpg: PSNR=13.43, SSIM=0.279, Entropy=7.92
Processed a0436-IMG_2583.jpg: PSNR=15.31, SSIM=0.714, Entropy=7.83
Processed a0452-IMG_1646.jpg: PSNR=15.68, SSIM=0.667, Entropy=6.64
Processed a0463-jmac_DSC2316.jpg: PSNR=15.81, SSIM=0.522, Entropy=7.14
Processed a0631-NKIM_MG_6442.jpg: PSNR=24.63, SSIM=0.699, Entropy=7.40
Processed a0648-IMG_5085.jpg: PSNR=15.36, SSIM=0.603, Entropy=7.82
Processed a0712-_DSC8911.jpg: PSNR=14.71, SSIM=0.672, Entropy=7.61
Processed a0752-20061213_134314__MG_3708.jpg: PSNR=20.75, SSIM=0.570, Entropy=7.24
Processed a0767-jn_20070824_0165.jpg: PSNR=13.99, SSIM=0.295, Entropy=7.76
Processed a0770-050703_161554__I2E9266.jpg: PSNR=22.76, SSIM=0.687, Entropy=7.62
Processed a0775-kme_423.jpg: PSNR=11.81, SSIM=0.426, Entropy=7.92
Processed a0811-20051224_165428__MG_0953.jpg: PSNR=22.50, SSIM=0.897, Entropy=7.51


In [2]:
import matplotlib.pyplot as plt
# Compute Averages
average_values = metrics_df1.mean(numeric_only=True)
print("\nAverage Values of our approach:")
print(average_values)


Average Values of our approach:
My PSNR       15.579869
My SSIM        0.551605
My Entropy     7.605993
dtype: float64


In [3]:
#my approch
import os
import cv2
import numpy as np
import pandas as pd
from skimage.metrics import structural_similarity as ssim
from skimage.measure import shannon_entropy

import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.measure import shannon_entropy

def compute_metrics(original, enhanced):
    """
    Compute PSNR, SSIM, and Shannon Entropy between the original and enhanced images.
    
    Parameters:
        original (np.array): Original image (BGR, uint8).
        enhanced (np.array): Enhanced image (BGR, uint8).
    
    Returns:
        tuple: (PSNR, SSIM, Entropy)
    """
    # Ensure both images have the same size (resize enhanced if necessary)
    if original.shape != enhanced.shape:
        enhanced = cv2.resize(enhanced, (original.shape[1], original.shape[0]))
    
    # PSNR computation
    psnr_value = cv2.PSNR(original, enhanced)
    
    # Determine an appropriate win_size for SSIM based on the image dimensions.
    h, w = original.shape[:2]
    min_dim = min(h, w)
    
    # If the smallest dimension is less than 3 pixels, SSIM is unreliable.
    if min_dim < 3:
        ssim_value = np.nan
    else:
        # Choose a win_size that is odd and does not exceed min_dim.
        if min_dim < 7:
            win_size = min_dim if (min_dim % 2 == 1) else (min_dim - 1)
        else:
            win_size = 7  # default win_size for larger images
        
        # Compute SSIM using the computed win_size and specifying channel_axis=-1 for color images.
        try:
            ssim_value, _ = ssim(original, enhanced, full=True, channel_axis=-1, win_size=win_size)
        except ValueError as e:
            print(f"Error computing SSIM for image of size ({h}, {w}) with win_size={win_size}: {e}")
            ssim_value = np.nan

    # Compute Shannon entropy on the enhanced image.
    # Convert the enhanced image to grayscale.
    enhanced_gray = cv2.cvtColor(enhanced, cv2.COLOR_BGR2GRAY)
    entropy_value = shannon_entropy(enhanced_gray)
    
    return psnr_value, ssim_value, entropy_value



def process_folders(original_folder, enhanced_folder, output_csv=r"D:\mit-5k-subset\cidnet_enhancement_metrics_partd.csv"):
    valid_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')
    results = []

    for filename in os.listdir(original_folder):
        if filename.lower().endswith(valid_extensions):
            orig_path = os.path.join(original_folder, filename)

            # Remove 'normal' prefix and add 'low' prefix
            basename = filename.replace('normal', 'low', 1)  # replace only the first occurrence
            enh_path = os.path.join(enhanced_folder, basename)

            if not os.path.exists(enh_path):
                print(f"Enhanced image for {filename} not found (expected {basename}). Skipping.")
                continue

            original_img = cv2.imread(orig_path)
            enhanced_img = cv2.imread(enh_path)

            if original_img is None or enhanced_img is None:
                print(f"Error reading {filename}. Skipping.")
                continue

            psnr_val, ssim_val, entropy_val = compute_metrics(original_img, enhanced_img)
            results.append({
                'Filename': filename,  # you can also use a common name like basename if you prefer
                'CIDNet PSNR': psnr_val,
                'CIDNet SSIM': ssim_val,
                'CIDNet Entropy': entropy_val
            })
            print(f"Processed {filename}: PSNR={psnr_val:.2f}, SSIM={ssim_val:.3f}, Entropy={entropy_val:.2f}")

    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")
    return df




    # Replace these with the paths to your folders.
original_folder = r"D:\mit-5k-subset\raw"
enhanced_folder = r"D:\mit-5k-subset\d_enhanced_cidnet"
    
    # Process the folders and save metrics to a CSV.
metrics_df2 = process_folders(original_folder, enhanced_folder)


Processed a0034-LSYD4O2202.jpg: PSNR=7.90, SSIM=0.156, Entropy=7.79
Processed a0035-dgw_048.jpg: PSNR=10.94, SSIM=0.430, Entropy=7.56
Processed a0291-IMG_0115.jpg: PSNR=9.44, SSIM=0.239, Entropy=7.69
Processed a0436-IMG_2583.jpg: PSNR=5.90, SSIM=0.449, Entropy=6.36
Processed a0452-IMG_1646.jpg: PSNR=7.93, SSIM=0.556, Entropy=5.85
Processed a0463-jmac_DSC2316.jpg: PSNR=9.41, SSIM=0.440, Entropy=7.02
Processed a0631-NKIM_MG_6442.jpg: PSNR=9.47, SSIM=0.611, Entropy=5.29
Processed a0648-IMG_5085.jpg: PSNR=8.01, SSIM=0.465, Entropy=6.79
Processed a0712-_DSC8911.jpg: PSNR=6.48, SSIM=0.461, Entropy=6.91
Processed a0752-20061213_134314__MG_3708.jpg: PSNR=8.46, SSIM=0.443, Entropy=6.29
Processed a0767-jn_20070824_0165.jpg: PSNR=8.18, SSIM=0.210, Entropy=7.70
Processed a0770-050703_161554__I2E9266.jpg: PSNR=8.31, SSIM=0.602, Entropy=5.22
Processed a0775-kme_423.jpg: PSNR=5.30, SSIM=0.246, Entropy=6.93
Processed a0811-20051224_165428__MG_0953.jpg: PSNR=10.39, SSIM=0.818, Entropy=4.56
Processed a0

In [4]:
import matplotlib.pyplot as plt
# Compute Averages
average_values = metrics_df2.mean(numeric_only=True)
print("\nCIDnet Average Values:")
print(average_values)


CIDnet Average Values:
CIDNet PSNR       8.245686
CIDNet SSIM       0.436387
CIDNet Entropy    6.691151
dtype: float64


In [5]:
merged_df = pd.merge(metrics_df1, metrics_df2, on='Filename', how='inner') 



In [6]:
merged_df.head(8)

,Filename,My PSNR,My SSIM,My Entropy,CIDNet PSNR,CIDNet SSIM,CIDNet Entropy
0,a0034-LSYD4O2202.jpg,12.546714,0.271476,7.871197,7.904240,0.155659,7.791245
1,a0035-dgw_048.jpg,12.281439,0.466825,7.555648,10.939869,0.429636,7.561438
2,a0291-IMG_0115.jpg,13.426651,0.278702,7.916655,9.435398,0.238918,7.692627
3,a0436-IMG_2583.jpg,15.306039,0.713682,7.830464,5.898341,0.449137,6.356144
4,a0452-IMG_1646.jpg,15.676783,0.666737,6.636784,7.930203,0.555894,5.852146
5,a0463-jmac_DSC2316.jpg,15.808558,0.522118,7.136401,9.407516,0.439778,7.021574
6,a0631-NKIM_MG_6442.jpg,24.628204,0.699274,7.396112,9.470929,0.611025,5.285074
7,a0648-IMG_5085.jpg,15.361514,0.602957,7.818565,8.007682,0.465343,6.793439


In [8]:
merged_df.to_csv("D:\mit-5k-subset\c_enhanced_cidnet/final_partd.csv")